# Coupled oscillators

An implementation of
"Task-dynamics of gestural timing: Phase windows and multifrequency rhythms"
by Saltzman & Byrd.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.rc('figure', figsize=(10, 5))

In [ ]:
# Helpers
def unwrap(angle):
    return np.mod(angle + np.pi, 2 * np.pi) - np.pi

def dangle(angle1, angle2, m=1, n=1):
    return unwrap(m * angle2 - n * angle1)

In [ ]:
def test_dangle():
    assert np.allclose(dangle(0.1, 0.2+np.pi*2), 0.1)
    assert np.allclose(dangle(0.1, 0.2-np.pi*2), 0.1)
    assert np.allclose(dangle(0.1+np.pi*2, 0.2), 0.1)
    assert np.allclose(dangle(0.1-np.pi*2, 0.2), 0.1)

    assert np.allclose(dangle(0.2, 0.1), -0.1)

    assert np.allclose(dangle(0.2+np.pi*2, 0.1), -0.1)
    assert np.allclose(dangle(0.2-np.pi*2, 0.1), -0.1)
    assert np.allclose(dangle(0.2, 0.1+np.pi*2), -0.1)
    assert np.allclose(dangle(0.2, 0.1-np.pi*2), -0.1)
    
    assert np.allclose(dangle(0, 0), 0)
    assert np.allclose(dangle(0.1, 0.1), 0)
    assert np.allclose(dangle(np.pi, -np.pi), 0)
    assert np.allclose(dangle(np.pi, 3*np.pi), 0)

test_dangle()

In [ ]:
class Oscillator(object):
    def __init__(self, init_phase=0, init_amplitude=2, freq=1, escape=1):
        self.alpha = -freq * escape
        self.beta = freq * escape
        self.gamma = float(escape) / freq
        self.freq = freq
        # track internal state in Cartesian coordinates; y = dx / freq
        self.x, self.dx = self.pol2cart(init_phase, init_amplitude)

    def pol2cart(self, angle, amplitude=1):
        x = amplitude * np.cos(angle)
        y = amplitude * np.sin(angle)
        dx = y * self.freq
        return x, dx

    def cartwrtpol(self):
        return self.freq * self.amplitude() * np.cos(self.angle())

    def amplitude(self):
        # Radial amplitude (i.e., A)
        return np.sqrt(self.x ** 2 + (self.dx / self.freq) ** 2)
        
    def angle(self):
        # Phase angle (i.e., phi)
        return -np.arctan2(self.dx / self.freq, self.x)

    def ddx(self):
        # Intrinsic oscillator dynamics
        return (-self.alpha * self.dx
                - self.beta * self.x ** 2 * self.dx
                - self.gamma * self.dx ** 3
                - self.freq ** 2 * self.x)

    def step(self, dt, task_ddx):
        # Simple Euler integration... could do something better
        self.dx += (self.ddx() + task_ddx) * dt
        self.x += self.dx * dt


class Sim(object):
    def __init__(self, init_relphase, desired=0, w1=1, w2=1, m=1, n=1, f1=1, f2=1, dt=0.1):
        assert -np.pi < init_relphase < np.pi, "Convert to radians"
        assert dt <= 0.5, "Unstable with dt > 0.5"
        
        self.jacobian = np.array([[-n, m]])
        self.w = np.array([[w1, 0], [0, w2]])
        self.jacobian_pinv = np.linalg.pinv(np.dot(self.jacobian, self.w))
        self.m = m
        self.n = n
        
        phase1 = 0.0
        phase2 = unwrap(-init_relphase)
        self.osc1 = Oscillator(init_phase=phase1, freq=f1)
        self.osc2 = Oscillator(init_phase=phase2, freq=f2)
        
        self.dt = dt
        self.desired = desired
        
        self.n_steps = 0
        self._relphases = [init_relphase]
        self._x1 = [self.osc1.x]
        self._x2 = [self.osc2.x]

    def trange(self, dt=None):
        dt = self.dt if dt is None else dt
        n_steps = int(self.n_steps * (self.dt / dt))
        return dt * np.arange(n_steps + 1)

    @property
    def relphases(self):
        return np.asarray(self._relphases)

    @property
    def x1(self):
        return np.asarray(self._x1)

    @property
    def x2(self):
        return np.asarray(self._x2)

    def dphase(self, phasediff):
        raise NotImplementedError('Implement in subclass')

    def task_ddx(self, phasediff):
        dphase = self.dphase(phasediff)
        dcart = np.array([self.osc1.cartwrtpol(), self.osc2.cartwrtpol()])
        return dcart * self.jacobian_pinv.ravel() * dphase

    def run(self, time_in_seconds):
        steps = int(np.round(float(time_in_seconds) / self.dt))
        for _ in xrange(steps):
            task = self.task_ddx(self._relphases[-1])
            self.osc1.step(self.dt, task[0])
            self.osc2.step(self.dt, task[1])
            self._x1.append(self.osc1.x)
            self._x2.append(self.osc2.x)
            self._relphases.append(dangle(self.osc1.angle(), self.osc2.angle(), m=self.m, n=self.n))
        self.n_steps += steps


class PointSim(Sim):
    def __init__(self, *args, **kwargs):
        self.a = kwargs.pop('a', 1)
        super(PointSim, self).__init__(*args, **kwargs)

    def dphase(self, phasediff):
        return self.a * np.sin(phasediff)


class SquashedSim(Sim):
    def __init__(self, *args, **kwargs):
        self.squashing = kwargs.pop('squashing', 4)
        super(SquashedSim, self).__init__(*args, **kwargs)

    def dphase(self, phasediff):
        z = -(np.cos(phasediff) + 1)
        return ((2 * self.squashing * np.exp(-self.squashing * z))
                / (1 + np.exp(-self.squashing * z)) ** 2
                * np.sin(phasediff))

In [ ]:
relphase = np.deg2rad(np.linspace(-180, 180, 200))

# Figure 2 left
def V(relphase, desired=0.0, a=1):
    return -a * np.cos(relphase - desired)

plt.plot(relphase, V(relphase), label="No window func")

# Figure 4 left
def V(relphase, desired=0.0, squashing=4, a=1):
    z = -(np.cos(relphase - desired) + 1)
    return 2. / (1 + np.exp(-squashing * z))

plt.plot(relphase, V(relphase), label="Squashed window")

plt.legend(loc="best")
plt.xlim(relphase[0], relphase[-1])

In [ ]:
relphase = np.deg2rad(140)
desired = np.deg2rad(0)
dt = 0.01

# Figure 2 right
sim = PointSim(relphase, desired, dt=dt)
sim.run(30)
plt.plot(sim.trange(), np.rad2deg(sim.relphases))

# Figure 4 right
sim = SquashedSim(relphase, desired, dt=dt)
sim.run(30)
plt.plot(sim.trange(), np.rad2deg(sim.relphases))

In [ ]:
relphase = np.deg2rad(140)
desired = np.deg2rad(0)
dt = 0.01

# Similar to figure 6, but with 1:1
sim = PointSim(relphase, desired, dt=dt)
sim.run(30)
plt.subplot(1, 2, 1)
plt.plot(sim.trange(), np.rad2deg(sim.x1))
plt.plot(sim.trange(), np.rad2deg(sim.x2))

# Similar to figure 6, but with 1:1
sim = SquashedSim(relphase, desired, dt=dt)
sim.run(30)
plt.subplot(1, 2, 2)
plt.plot(sim.trange(), np.rad2deg(sim.x1))
plt.plot(sim.trange(), np.rad2deg(sim.x2))

In [ ]:
relphase = np.deg2rad(140)

def fig6(m, n, desired):
    # I think f1, f2 should be n, m but the paper disagrees...
    sim = PointSim(relphase, desired, m=m, n=n, f1=m, f2=n, dt=dt)
    sim.run(20)
    plt.plot(sim.trange(), np.rad2deg(sim.x1))
    plt.plot(sim.trange(), np.rad2deg(sim.x2))

# Figure 6 a
plt.figure()
fig6(2, 1, np.deg2rad(0))

# Figure 6 b
plt.figure()
fig6(3, 2, np.deg2rad(180))

# Figure 6 c
plt.figure()
fig6(4, 3, np.deg2rad(90))

In [ ]:
relphase = np.deg2rad(140)

def fig7(w1, w2):
    sim = PointSim(relphase, desired, m=2, n=1, f1=w1, f2=w2, dt=dt)
    sim.run(40)
    plt.subplot(2, 1, 1)
    plt.plot(sim.trange(), np.rad2deg(sim.x1))
    plt.plot(sim.trange(), np.rad2deg(sim.x2))
    plt.subplot(2, 1, 2)
    plt.plot(sim.trange(), np.rad2deg(sim.relphases))

# Figure 7 a
plt.figure()
fig7(2, 1)

# Figure 7 b
plt.figure()
fig7(2, 1.25)

# Figure 7 c
plt.figure()
fig7(2, 1.39)

# Figure 7 d
plt.figure()
fig7(2, 1.9)